# Importação de bibliotecas

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from rapidfuzz import fuzz
from tqdm import tqdm
import unicodedata
import string
from datetime import datetime
from joblib import Parallel, delayed

In [3]:
# Download das stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')

# Removendo stopwords específicas
# Removido 'com' e 'sem' pq pode ter bicicleta com e sem marcha
# Removido 'pelo' pois pode ser refetente ao capilar
stopwords.remove('com')
stopwords.remove('sem')
stopwords.remove('pelo')
stopwords_upper = [x.upper() for x in stopwords]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Evelyn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
stopwords

['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'às',
 'até',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'é',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'éramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'está',
 'estamos',
 'estão',
 'estar',
 'estas',
 'estava',
 'estavam',
 'estávamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estivéramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéssemos',
 'estou',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'fôramos',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fôssemos',
 'fui',
 'há',
 'haja',
 'hajam',
 'hajamos',
 'hão',
 'havemos',
 'haver',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houverá',
 'houveram',
 'houvéramos',
 'houverão',
 'houverei',
 'houverem',
 'houveremos'

# Carregamento de dados

In [5]:
# Capturando o timestamp para o nome do arquivo
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Carregar os DataFrames
df = pd.read_csv('items_titles_test.csv')
df_test = pd.read_csv('items_titles.csv')

# Pré-processamento

In [6]:
def remove_accent(string):
    nfkd_form = unicodedata.normalize('NFKD', string)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])


def process_string(s):
    s = s.upper()
    s = re.sub(r'[' + string.punctuation + ']', '', s)  # Remove symbols
    s = remove_accent(s)
    words = s.split()
    words = [word for word in words if word not in stopwords_upper]  # Remove stopwords
    return ' '.join(words)


Antes de aplicar o processamento nos dados, faço uma cópia para a coluna "name_copia"
depois aplico regras como upper, remoção de símbolos, remoção de acentuação, e remoção de stopwords

In [7]:
df['name_copia'] = df['ITE_ITEM_TITLE'].apply(process_string)
df

,ITE_ITEM_TITLE,name_copia
0,Tênis Olympikus Esporte Valente - Masculino Kids,TENIS OLYMPIKUS ESPORTE VALENTE MASCULINO KIDS
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,BICICLETA BARRA FORTE SAMY C 6 MARCHAS CUBO C ...
2,Tênis Usthemp Slip-on Temático - Labrador 2,TENIS USTHEMP SLIPON TEMATICO LABRADOR 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye,TENIS CASUAL FEMININO MOLECA TECIDO TIE DYE
4,Tênis Star Baby Sapatinho Conforto + Brinde,TENIS STAR BABY SAPATINHO CONFORTO BRINDE
...,...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil,CHUTEIRA FUTSAL OXN VELOX 3 INFANTIL
9996,Sapatenis Casual Masculino Estiloso 24horas Co...,SAPATENIS CASUAL MASCULINO ESTILOSO 24HORAS CO...
9997,Tênis Feminino Infantil Molekinha Tie Dye,TENIS FEMININO INFANTIL MOLEKINHA TIE DYE
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...,TENIS FEMININO LEVE BARATO GANHE 1 COLCHONETE ...


In [8]:
df_test['name_copia'] = df_test['ITE_ITEM_TITLE'].apply(process_string)
df_test

,ITE_ITEM_TITLE,name_copia
0,Tênis Ascension Posh Masculino - Preto E Verme...,TENIS ASCENSION POSH MASCULINO PRETO VERMELHO
1,Tenis Para Caminhada Super Levinho Spider Corr...,TENIS CAMINHADA SUPER LEVINHO SPIDER CORRIDA
2,Tênis Feminino Le Parc Hocks Black/ice Origina...,TENIS FEMININO LE PARC HOCKS BLACKICE ORIGINAL...
3,Tênis Olympikus Esportivo Academia Nova Tendên...,TENIS OLYMPIKUS ESPORTIVO ACADEMIA NOVA TENDEN...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...,INTELIGENTE LED BICICLETA TAUDA LUZ USB BICICL...
...,...,...
29995,Tênis Vans Old Skool I Love My Vans - Usado - ...,TENIS VANS OLD SKOOL I LOVE MY VANS USADO FEMI...
29996,Tênis Feminino Preto Moleca 5296155,TENIS FEMININO PRETO MOLECA 5296155
29997,Tenis Botinha Com Pelo Via Marte Original Lanç...,TENIS BOTINHA COM PELO VIA MARTE ORIGINAL LANC...
29998,Tênis Slip On Feminino Masculino Original Sapa...,TENIS SLIP ON FEMININO MASCULINO ORIGINAL SAPA...


## Cálculo de similaridade

In [19]:
# Função de cálculo de similaridade
def calculate_similarity(title1, title2):
    # Verifico se o comprimento de title1 é menor que 3 ou se o comprimento de title2 é muito maior que title1
    if (len(title1) <= 3 and len(title2) > 2 * len(title1)) or (len(title2) <= 3 and len(title1) > 2 * len(title2)):
        return 0.0
    if not title1 or not title2:
        return 0.0
    if title1 == title2:
        return 1.0
    return fuzz.ratio(title1, title2) / 100


# Função para computar scores paralelamente
# Faz a comparação com o campo tratato "name_copia" mas armazena o ITE_ITEM_TITLE nos resultados
# Ao invés de For aninhado, utilizado o pandas, o processo foi mais rápido
def compute_scores(row_test):
    local_results = []
    scores = df['name_copia'].apply(lambda x: calculate_similarity(row_test['name_copia'], x))
    for idx, score in scores.items():
        local_results.append([row_test['ITE_ITEM_TITLE'], df.at[idx, 'ITE_ITEM_TITLE'], score])
    return local_results

## Calculo de Scores paralelos

In [29]:
# Utilizando joblib para paralelizar o cálculo de similaridade

results = Parallel(n_jobs=-1)(delayed(compute_scores)(row_test) for _, row_test in tqdm(df_test.head(100).iterrows(), total=min(100, df_test.shape[0]), desc="Processing df_test")) # Rodar amostra dos 50 primeiros itens

#results = Parallel(n_jobs=-1)(delayed(compute_scores)(row_test) for _, row_test in tqdm(df_test.iterrows(), total=df_test.shape[0], desc="Processing df_test")) #Rodar Tudo

print(f"Processado {df.shape[0]} registros em df")
print(f"Processado {df_test.shape[0]} registros em df_test")
print(f"Finalizado")

Processing df_test: 543it [02:24,  3.75it/s]
Processing df_test: 100%|██████████| 100/100 [00:03<00:00, 25.29it/s]


Processado 10000 registros em df
Processado 30000 registros em df_test
Finalizado


## Achatamento de listas

Como a saída da função paralela é uma lista de listas, eh necessário "achatar" a lista para ter uma lista única.

In [30]:
results = [item for sublist in results for item in sublist]
results

[['Tênis Ascension Posh Masculino - Preto E Vermelho ',
  'Tênis Olympikus Esporte Valente - Masculino Kids',
  0.4835164835164834],
 ['Tênis Ascension Posh Masculino - Preto E Vermelho ',
  'Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C/ Rolamento',
  0.38],
 ['Tênis Ascension Posh Masculino - Preto E Vermelho ',
  'Tênis Usthemp Slip-on Temá\x81tico - Labrador 2',
  0.48837209302325574],
 ['Tênis Ascension Posh Masculino - Preto E Vermelho ',
  'Tênis Casual Feminino Moleca Tecido Tie Dye',
  0.5227272727272727],
 ['Tênis Ascension Posh Masculino - Preto E Vermelho ',
  'Tênis Star Baby Sapatinho Conforto + Brinde',
  0.48837209302325574],
 ['Tênis Ascension Posh Masculino - Preto E Vermelho ',
  'Tênis Oakley Frequency 3.0 Preto/marrom',
  0.46341463414634143],
 ['Tênis Ascension Posh Masculino - Preto E Vermelho ',
  'Tênis Jogging Feminino Premium Super Lançamento Vizzano',
  0.46],
 ['Tênis Ascension Posh Masculino - Preto E Vermelho ',
  'Under Armour Hovr Phantom 2 Conexão Blu

# Resultados

In [31]:
# Criando o DataFrame de resultados
df_results = pd.DataFrame(results, columns=['ITE_ITEM_TITLE_1', 'ITE_ITEM_TITLE_2', 'Score Similituud'])
df_results = df_results.sort_values(by='Score Similituud', ascending=False)
df_results.head(10)

,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,Score Similituud
513038,Tenis Infantil Preto Molekinho/ Dia Das Crianças,Tenis Infantil Preto Molekinho/ Dia Das Crianças,1.000000
756407,Tênis Anacapri Branco Corda Flatform Original,Tênis Anacapri Branco Corda Flatform Original,1.000000
122828,Kit Tênis Slip Feminino Com Mochila Grande E M...,Kit Tênis Slip Feminino Com Mochila Grande E M...,1.000000
321333,Lubrificante Lube Cera 25g,Lubrificante Lube Cera 25g,1.000000
602037,Ténis Estilo All Star Cano Alto Masculino De C...,Ténis Estilo All Star Cano Alto Masculino De C...,0.960000
87432,Tênis Usthemp Short Temático - Maria Vira-lata 2,Tênis Usthemp Half Temático - Maria Vira-lata,0.898876
609920,Ténis Estilo All Star Cano Alto Masculino De C...,Ténis Estilo All Star Cano Alto Masculino Em L...,0.888889
842062,Tênis Puma Softride Rift Tech Feminino,Tênis Puma Softride Rift Tech Feminino - Preto...,0.873563
89618,Tênis Usthemp Short Temático - Maria Vira-lata 2,Tênis Usthemp Long Temático - Armani Vira-lata 2,0.869565
210904,Tenis Feminino De Fazer Caminhada Corrida Acad...,Tênis Feminino Caminhada Corrida Academia,0.863158


In [32]:
# Salvando o arquivo tanto em csv qnto em pikle
filename = f"similarity_results_{timestamp}"
df_results.to_csv(f"{filename}.csv", index=False)
df_results.to_pickle(f"{filename}.pkl")

In [38]:
df_results.head()


,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,Score Similituud
513038,Tenis Infantil Preto Molekinho/ Dia Das Crianças,Tenis Infantil Preto Molekinho/ Dia Das Crianças,1.00
756407,Tênis Anacapri Branco Corda Flatform Original,Tênis Anacapri Branco Corda Flatform Original,1.00
122828,Kit Tênis Slip Feminino Com Mochila Grande E M...,Kit Tênis Slip Feminino Com Mochila Grande E M...,1.00
321333,Lubrificante Lube Cera 25g,Lubrificante Lube Cera 25g,1.00
602037,Ténis Estilo All Star Cano Alto Masculino De C...,Ténis Estilo All Star Cano Alto Masculino De C...,0.96


In [37]:
df_results.tail()

,ITE_ITEM_TITLE_1,ITE_ITEM_TITLE_2,Score Similituud
594423,Tênis Skechers Go Walk Joy Marinho,Bom,0.0
41646,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...,Ês,0.0
674423,Tênis Olympikus Pride 2 Feminino Verde E Rosa,Bom,0.0
264423,Tênis Antiviral Piccadilly Slip-on Sem Cadarço...,Bom,0.0
69504,Tênis Infantil Ortopasso Conforto Jogging,Bmx,0.0


In [35]:
print(f"Number of rows in df: {df.shape[0]}")

Number of rows in df: 10000


In [36]:
print(df_results.describe())

       Score Similituud
count    1000000.000000
mean           0.406489
std            0.075426
min            0.000000
25%            0.355556
50%            0.407767
75%            0.454545
max            1.000000


Para este desafio, o número de possibilidades geradas é de n * m, ou seja, 30k * 10k das listas, visto que o teste solicitou todas as combinações com os devidos scores, o resultado é de 300 mihoes de linhas.

Para fins de demonstração, foi utilizado uma pequena amostra de teste com 100 linhas em uma lista e 10mil em outra, gerando o resultado em 2min com o paralelismo.

Em um próximo momento, faz sentido explorar a similaridade de cosseno para só calcular score para as palavras que fazem algum sentido.

